In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import sys
sys.path.append('..')

In [21]:
import os

import torch
from torchsummary import summary
import torchaudio
from IPython.display import Audio
from scipy.io import wavfile 
import librosa

In [7]:
from dataset import *
from cnn import CNNetwork

In [42]:
if torch.cuda.is_available():
        device = "cuda"
else:
        device = "cpu"
print(f"Using device {device}")

Using device cpu


In [9]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=16000,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )
dataset = VoiceDataset('../data/train', mel_spectrogram, 16000, device)

In [10]:
len(dataset)

5717

In [11]:
dataset[0]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
dataset[0][0].shape

In [12]:
cnn = CNNetwork()
# summary(cnn, (1, 64, 44))

In [13]:
torch.tensor(0)

tensor(0)

In [14]:
dataset.label_mapping

{'aman': 0, 'imran': 1, 'labib': 2}

In [15]:
dataset[0]

AssertionError: Torch not compiled with CUDA enabled

In [16]:
from datetime import datetime
now = datetime.now()

In [17]:
now.strftime("%Y%m%d_%H%M%S")

'20230516_095454'

In [18]:
cnn.load_state_dict(torch.load("../models/void_20230512_225714.pth"))

RuntimeError: Error(s) in loading state_dict for CNNetwork:
	size mismatch for linear.weight: copying a param with shape torch.Size([3, 7040]) from checkpoint, the shape in current model is torch.Size([3, 35712]).

In [ ]:
x, y = dataset[10]

In [152]:
labels = dataset._labels

In [153]:
input = x.unsqueeze_(0) 

In [182]:
def predict(model, input, target, class_mapping):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
        predicted_index = predictions[0].argmax(0)
        predicted = class_mapping[predicted_index]
        expected = class_mapping[target]
    return predictions

In [155]:
predict(cnn, input, y, labels)

tensor([[1.0000e+00, 1.3728e-20, 2.8026e-44]])


('aman', 'aman')

In [156]:
input

tensor([[[[0.0259, 0.1384, 0.0784,  ..., 0.0000, 0.0000, 0.0000],
          [0.0334, 0.1320, 0.0701,  ..., 0.0000, 0.0000, 0.0000],
          [0.0481, 0.0324, 0.0545,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.2665, 0.3647, 0.3147,  ..., 0.0000, 0.0000, 0.0000],
          [0.2710, 0.3796, 0.2160,  ..., 0.0000, 0.0000, 0.0000],
          [0.1950, 0.2607, 0.1905,  ..., 0.0000, 0.0000, 0.0000]]]])

In [157]:
input.shape

torch.Size([1, 1, 64, 157])

In [158]:
x.shape

torch.Size([1, 1, 64, 157])

In [159]:
os.path.join('data', 'aman', 'aman_1')

'data/aman/aman_1'

In [45]:
test_dataset =  VoiceDataset('../data/test', mel_spectrogram, device)

In [46]:
test_dataset.audio_files_labels

[('aman_1.wav', 'aman'),
 ('aman_3.wav', 'aman'),
 ('aman_2.wav', 'aman'),
 ('aman_6.wav', 'aman'),
 ('aman_7.wav', 'aman'),
 ('aman_5.wav', 'aman'),
 ('aman_4.wav', 'aman'),
 ('imran_4.wav', 'imran'),
 ('imran_5.wav', 'imran'),
 ('imran_6.wav', 'imran'),
 ('imran_2.wav', 'imran'),
 ('imran_3.wav', 'imran'),
 ('imran_1.wav', 'imran'),
 ('labib_1.wav', 'labib'),
 ('labib_3.wav', 'labib'),
 ('labib_2.wav', 'labib'),
 ('labib_6.wav', 'labib'),
 ('labib_5.wav', 'labib'),
 ('labib_4.wav', 'labib')]

In [47]:
test_input, test_output = test_dataset[0]
test_input.shape

torch.Size([1, 64, 469])

48000

In [179]:
test_input = test_input.unsqueeze_(0) 

In [180]:
test_input.shape

torch.Size([1, 1, 64, 157])

In [185]:
output = predict(cnn, test_input, test_output, test_dataset._labels)

tensor([[1., 0., 0.]])


In [188]:
pred = torch.argmax(output, 1)

In [19]:
torch.load("https://www2.cs.uic.edu/~i101/SoundFiles/BabyElephantWalk60.wav")

FileNotFoundError: [Errno 2] No such file or directory: 'https://www2.cs.uic.edu/~i101/SoundFiles/BabyElephantWalk60.wav'

In [20]:
import tempfile

In [21]:
temp = tempfile.TemporaryFile()

In [26]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=67b26307ba20670a602d8355a077eb17f9f0a53530002bc0f5bc406c82dda766
  Stored in directory: /Users/amanmibra/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [1]:
import wget

In [35]:
filename = wget.download("https://www2.cs.uic.edu/~i101/SoundFiles/BabyElephantWalk60.wav")

100% [..........................................................................] 2646044 / 2646044

In [30]:
filename

'BabyElephantWalk60.wav'

In [37]:
torchaudio.load(filename)

(tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0125, -0.0136, -0.0661]]), 22050)

In [38]:
os.remove(filename)

In [36]:
os.path.exists(filename)

True

In [48]:
import requests

In [49]:
size = (1, 128, 469)

In [51]:
fake_wav = torch.rand(size)

In [59]:
requests.put("https://localhost:8000/predict", data={"wav": fake_wav})

SSLError: HTTPSConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /predict (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))

In [48]:
filename = wget.download("https://cdn.filestackcontent.com/eWof5DcWRKGLO1OjDNkG")

100% [..............................................................................] 18421 / 18421

In [49]:
filename

'temp'

In [51]:
audio, sr = librosa.load(filename)

In [52]:
Audio(audio, rate=sr)

In [53]:
wavfile.write('temp.wav', sr, audio)

In [45]:
audio, sr = torchaudio.load('temp.wav')

In [46]:
Audio(audio, rate=sr)

In [47]:
os.remove('temp')
os.remove('temp.wav')
